In [1]:
import os
import sys
import random
import json
import itertools
import importlib
import pandas as pd
import nibabel as nib
from datetime import datetime

import visUtils
import stitchSurfs as stitch
import projectUtils as prjUtils

sys.path.append('/host/verges/tank/data/daniel/00_commonUtils/00_code/genUtils/')
#print("Contents:", os.listdir('/host/verges/tank/data/daniel/00_commonUtils/00_code/genUtils/'))

import gen
import bids_naming as names
import dataChecks as check

In [2]:
# parameters

test = True
test_n = 2
verbose = True

demographics_pth = '/host/verges/tank/data/daniel/00_commonUtils/01_demographics/02_combined/demographics_23Jan2026-161349.csv'

dirs_project = {
    'dir_root': '/host/verges/tank/data/daniel/04_inVivoHistology/',
    'dir_data': 'data/',
    'dir_out': 'outputs/'
}

features = ["T1map"]

pni_resolution = (0.5, 0.5, 0.5)  # target resolution for PNI 7T scans

lbls_surfs = { # [ctx, hipp]
    'labels': [['pial', 'inner'],
               ['white', 'outer']],
    'surfaces': [['fsLR-32k', 'den-0p5mm']]
}

PNI = {
    'studyName': 'PNI',
    'studyDescrip': '7T',
    'dir_root': '/data/mica3/BIDS_PNI/',
    'dir_deriv': 'derivatives/',
    'dir_mp': 'micapipe_v0.2.0/',
    'dir_hu': 'hippunfold_v1.3.0/hippunfold/', # update to v2?
}

MICS = {
    'studyName': 'MICs',
    'studyDescrip': '3T',
    'dir_root': '/data/mica3/BIDS_MICs/',
    'dir_deriv': 'derivatives/',
    'dir_mp': 'micapipe_v0.2.0/',
    'dir_hu': 'hippunfold_v1.3.0/hippunfold/', # update to v2?
}

varsOfInterest = ['UID', 'MICS_ID', 'PNI_ID', 'study', 'SES', 'Date', 'sex', 'age', 'grp','grp_detailed']

study_dicts = [PNI, MICS]


In [3]:
# input: demographics datasheet
demo = pd.read_csv(demographics_pth)
print("Shape of demographics:", demo.shape)
print("Columns in demographics:", demo.columns.tolist())
demo.head()

Shape of demographics: (871, 48)
Columns in demographics: ['UID', 'MICS_ID', 'PNI_ID', 'study', 'SES', 'Date', 'gender', 'sex', 'employment', 'language', 'dob', 'education', 'ethnicity', 'lastSeizure', 'handedness', 'Scan_Date (D.M.Y)', 'WeightApprox', 'Handed', 'Employ', 'YoE', 'AssignedSex', 'HeightApprox', 'ASMs  on admission (name, doses (mg per day)', 'Epilepsy diagnosis based on ILAE', 'FDG.PET', 'Duration of admission', 'Surgical resection date and site', 'Invasive explorations (Y/N)', 'Dx at EMU discharge ', 'Baseline MRI (year,results)', 'Epilepsy classification:Focal,Generalized', 'Lateralization of epileptogenic focus', 'Engel classification (seizure outcomes at the 6 month )', 'Epileptogenic focus based on EMU information', 'Drug resistant epilepsy at time of EMU admission', 'Histopatholgy', 'Previous ASMs (name and doses (mg/d)) if applicable prior the current EMU admission', '# of ASM on admission', 'Risk factors for epilepsy', '# of ASMs prior current EMU admission', 'IL

,UID,MICS_ID,PNI_ID,study,SES,Date,gender,sex,employment,language,...,Risk factors for epilepsy,# of ASMs prior current EMU admission,ILAE outcome after surgical resection by 1 yr,Engel classification (seizure outcomes after 1 year from surgical resection),Neuromodulation devices,EMU admission date(dd-mm-yy),# of surgical resection/thermocoagulatin,age,grp,grp_detailed
0,UID0001,NaN,Pilot007,7T,00,11.03.2022,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTRL,CTRL
1,UID0002,NaN,Pilot011,7T,05,28.03.2024,NaN,NaN,Full time student,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTRL,CTRL
2,UID0003,NaN,Pilot012,7T,05,11.04.2024,NaN,NaN,Full time student,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTRL,CTRL
3,UID0004,HC129,Pilot013,7T,05,18.04.2024,NaN,F,Full time student,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.728953,CTRL,CTRL
4,UID0004,HC129,Pilot013,3T,01,09.07.2024,NaN,F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.953457,CTRL,CTRL


In [4]:
# 7T only
do_checks = False
if do_checks:
    importlib.reload(gen)
    importlib.reload(names)
    importlib.reload(check)
    qc_cols = []

    sT_pt = demo[demo['PNI_ID'].isna()==False][varsOfInterest]
    sT_sessions = sT_pt[sT_pt['study'] == '7T']
    print(f'Number of 7T sessions: {len(sT_sessions)}')
    #print(sT_sessions.head())

    print("CHECKING DATA...")
    print(f"\t VOLUMES existence...")
    demo_volCheck, cols_volChk = check.vol_check(PNI, sT_sessions, features, verbose=verbose)
    qc_cols += cols_volChk

    print(f"\t RESOLUTION match {pni_resolution}...")
    demo_res, res_dictList, cols_resChk = check.resolution_check(demo = sT_sessions, study = PNI, res_trgt = pni_resolution, epsilon=0.001, verbose = verbose)
    qc_cols += cols_resChk

    print(f"\t SURFACES present...")
    demo_volSurf_check, cols_procChk = check.proc_check(study = PNI, demo = demo_volCheck, mp_surfaces = mp_surfaces, hu_surfaces = hu_surfaces, verbose=verbose)
    qc_cols += cols_procChk

    # check QC of surfaces and volumes
    # TODO.


    # save
    save_pth = os.path.join(dirs_project['dir_root'], dirs_project['dir_out'], f'demo_QCcols_{gen.fmt_now()}')
    save_item = [demo_volSurf_check, qc_cols]
    demo_volSurf_check.to_parquet(save_pth + ".parquet", index=False)
    # save list
    with open(f"{save_pth}.json", "w") as f:
        json.dump(qc_cols, f, indent=2)
    print(f"Saved DataFrame to: {save_pth}.parquet")
    print(f"Saved QC columns list to: {save_pth}.json")


In [5]:
reimport = False
if "demo_volSurf_check" not in locals() or reimport:
    common_pth = "/host/verges/tank/data/daniel/04_inVivoHistology/outputs/demo_QCcols_29Jan2026-1206"
    demo_volSurf_check = pd.read_parquet(f"{common_pth}.parquet")
    qc_cols = json.load(open(f"{common_pth}.json"))
    print(f"Loaded DataFrame from: {common_pth}.parquet")
    print(f"Loaded QC columns list from: {common_pth}.json")

# filter out bad QC
qc_cols_present = [col for col in qc_cols if col in demo_volSurf_check.columns]
demo_qc, rm_rows = check.filter_qcCols(demo_volSurf_check, qc_cols)

if test:
    # take random sample of test_n subjects
    demo_qc = demo_qc.sample(n=test_n)
    print(f"TEST MODE: {test_n} rows kept:")
    for idx, row in demo_qc.iterrows():
        print(f"\t{row['UID']}@{row['study']}: {row['MICS_ID']}-{row['PNI_ID']}-{row['SES']}")

demo_qc.head()

Loaded DataFrame from: /host/verges/tank/data/daniel/04_inVivoHistology/outputs/demo_QCcols_29Jan2026-1206.parquet
Loaded QC columns list from: /host/verges/tank/data/daniel/04_inVivoHistology/outputs/demo_QCcols_29Jan2026-1206.json
[filter_qcCols] 19 rows (17 unique participants) removed when filtering QC cols: ['hasVol_T1map', 'properRes', 'mp_proc_pial_fsLR-32k', 'mp_proc_white_fsLR-32k', 'hu_proc_inner_den-0p5mm', 'hu_proc_outer_den-0p5mm']
TEST MODE: 2 rows kept:
	UID0034@7T: None-PNC021-01
	UID0125@7T: PX256-PNE057-a1


,UID,MICS_ID,PNI_ID,study,SES,Date,sex,age,grp,grp_detailed,hasVol_T1map,properRes,mp_proc_pial_fsLR-32k,mp_proc_white_fsLR-32k,hu_proc_inner_den-0p5mm,hu_proc_outer_den-0p5mm
81,UID0034,None,PNC021,7T,01,12.06.2023,None,NaN,CTRL,CTRL,True,1.0,True,True,True,True
197,UID0125,PX256,PNE057,7T,a1,03.12.2025,F,46.915811,TLE,TLE_BL,True,1.0,True,True,True,True


In [ ]:
# create sampling across cortical thickness (14 samples between pial/inner and white/outer)
# cf cortPro: https://github.com/caseypaquola/CortPro
# LAYNII: https://github.com/layerfMRI/LAYNII

In [8]:
# stitch surfaces
importlib.reload(prjUtils)
prjUtils.stitch_surfs_from_df(dirs_project = dirs_project, study_dicts = study_dicts, 
                              df = demo_qc, lbls_surfs = lbls_surfs)

[stitch_surfs_from_df] Stitching surfaces for 2 rows (unique participant-study-session)...
UID0034@7T: PNC021-01
	[stitch_surfs_from_df] Stitching [ctx] fsLR-32k_pial to [hipp] den-0p5mm_inner -> /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/sub-PNC021_ses-01_hemi_L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii | /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/sub-PNC021_ses-01_hemi_R_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii
Saved stitched surface to: /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/sub-PNC021_ses-01_hemi_L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii
Saved stitched surface to: /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/sub-PNC021_ses-01_hemi_R_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii
	[stitch_surfs_from_df] 

['/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/sub-PNC021_ses-01_hemi_L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii',
 '/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/sub-PNC021_ses-01_hemi_R_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii',
 '/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/sub-PNC021_ses-01_hemi_L_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched.surf.gii',
 '/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/sub-PNC021_ses-01_hemi_R_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched.surf.gii',
 '/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNE057_ses-a1/sub-PNE057_ses-a1_hemi_L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii',
 '/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNE057_se

In [ ]:
# mask out other cortical areas; do this before or after stitching?
# as per Paquola, only include neo cortex with the following areas:
neo_cortex_areas = ["entorhinal", "parahippocampal", "fusiform"] # check these label naming

freeview -f /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC021/ses-01/surf/sub-PNC021_ses-01_hemi-L_space-nativepro_surf-fsLR-32k_label-pial.surf.gii:edgecolor=yellow -f /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC021/ses-01/surf/sub-PNC021_ses-01_hemi-R_space-nativepro_surf-fsLR-32k_label-pial.surf.gii:edgecolor=yellow -f /data/mica3/BIDS_PNI/derivatives/hippunfold_v1.3.0/hippunfold/sub-PNC021/ses-01/surf/sub-PNC021_ses-01_hemi-L_space-T1w_den-0p5mm_label-hipp_inner.surf.gii:edgecolor=orange -f /data/mica3/BIDS_PNI/derivatives/hippunfold_v1.3.0/hippunfold/sub-PNC021/ses-01/surf/sub-PNC021_ses-01_hemi-R_space-T1w_den-0p5mm_label-hipp_inner.surf.gii:edgecolor=orange -f /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/sub-PNC021_ses-01_hemi_L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii:edgecolor=white -f /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/sub-PNC021_ses-01_hemi_R_c

In [ ]:
importlib.reload(visUtils)
importlib.reload(prjUtils)
importlib.reload(check)
# get bash command to visualize surfaces and volumes
add_stitchedSurf = True
for pt in demo_qc.itertuples():
    uid = pt.UID
    study = pt.study
    ses = pt.SES
    mics_id = pt.MICS_ID
    pni_id = pt.PNI_ID

    if study == '7T':
        id = pni_id
        study_dict = next(sd for sd in study_dicts if sd['studyName'] == 'PNI')
    elif study == '3T':
        id = mics_id
        study_dict = next(sd for sd in study_dicts if sd['studyName'] == 'MICs')
    else:
        print(f"[main] WARNING: study {study} not recognized. Skipping subject {uid}...")
        continue

    mp_root = study_dict['dir_root'] + study_dict['dir_deriv'] + study_dict['dir_mp']
    hu_root = study_dict['dir_root'] + study_dict['dir_deriv'] + study_dict['dir_hu']

    # get paths
    vol_pth = names.get_volPath(study = study_dict, id = id, ses = ses, volName = 'T1map')[0]
    
    if add_stitchedSurf:
        dir_subjectData = prjUtils.get_path_data(dirs_project, studyName = study_dict['studyName'], id = id, ses = ses)
        prjUtils.make_dir(dir_subjectData)

    surface_combinations = prjUtils.iterate_labels(lbls_surfs)

    surf_pths = []
    for surfs, lbls in surface_combinations:
        ctx_surf = surfs[0]
        hipp_surf = surfs[1]

        ctx_lbl = lbls[0]
        hipp_lbl = lbls[1]

        mp_surfs = names.get_surf_pth(root = mp_root, sub = id, ses = ses, lbl=ctx_lbl, surf=ctx_surf, verbose = False)
        hu_surfs = names.get_surf_pth(root = hu_root, sub = id, ses = ses, lbl=hipp_lbl, surf=hipp_surf, verbose = False)
        surf_pths += mp_surfs
        surf_pths += hu_surfs

        if add_stitchedSurf:
            stched_name_l, stched_name_r = prjUtils.get_names_stitchSurf(id=id, ses=ses, ctx_lbl=ctx_lbl, ctx_surf=ctx_surf, hipp_lbl=hipp_lbl, hipp_surf=hipp_surf)
            surf_pths += [os.path.join(dir_subjectData, stched_name_l), 
                          os.path.join(dir_subjectData, stched_name_r)]

    # generate visualization command
    # check that all files exist
    surf_exist = check.check_paths_exist(surf_pths)
    surf_pths = [surf_pth for surf_pth, exist in zip(surf_pths, surf_exist) if exist]
    print(visUtils.getCMD_freeView(surf_pths=surf_pths, vol_pth=vol_pth))



In [ ]:


# following steps should be able to be performed on data from any source (PNI, MICs, AHEAD, BigBrain, etc.)

# save surfaces generated at different depths to study directory
# stitch cortical and hippocampal surfaces together. NOTE. Jordan code
# remove vertices not in mesial temporal lobe (choose cut-off limit as per Paquola 2020)
# assign each vertex values along anterior-posterior, and allo/neo-cortical axes
# project features onto surfaces


In [ ]:
# analyses